<a href="https://colab.research.google.com/github/ANUJGAUR/NYC-Taxi-Trip-Time-Prediction---Capstone-Project/blob/main/NYC_Taxi_Trip_Time_Prediction_Capstone_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Project Name**    - 







##### **Project Type**    - Regression
##### **Contribution**    - Team
##### **Team Member 1 - Arunkumar Gond**
##### **Team Member 2 - Anuj Kumar Gaur**


# **Project Summary -**

**Introduction**

The New York City Taxi and Limousine Commission (TLC), created in 1971, is the agency responsible for licensing and regulating New York City's Medallion (Yellow) taxi cabs, for-hire vehicles (community-based liveries, black cars and luxury limousines), commuter vans, and paratransit vehicles. Over 200,000 TLC licensees complete approximately 1,000,000 trips each day.

**Problem statement**

Our task is to build a model that predicts the total ride duration of taxi trips in New York City. Our primary dataset is one released by the NYC Taxi and Limousine Commission, which includes pickup time, geo-coordinates, number of passengers, and several other variables.

**Approaches**

Initially we started understanding and analysing the data. After some initial analysing, we started data analysing. The data we have noticed that some columns like id and last review are not useful for us so we drop both the columns. Some columns and rows have null values which also need to removed or replaced. We also found outliners in some columns and rows, such as price column. We replaced those values with in a suitable price range. This whole process is called as Data Cleaning, Transformation and Analyses. After that we have applied different regression model on cleaned dataset and selected best model based on the R2 value.

**Conclusions**



*   We can see that MSE and R2 and Adjusted R2 which are the metrics used to 
evaluate the performance of regression model of Linear Regression, Decision Tree, Random Forest and XGBoost Regressor.
*  The Linear models don't show good performance on our training and testing environment.


*   From above table we can conclude that Random Forest Regressor (80%) is the best models as compare to the other models to predict the trip duration for a particular taxi.


# **GitHub Link -**

https://github.com/ANUJGAUR/NYC-Taxi-Trip-Time-Prediction---Capstone-Project

# **Problem Statement**


**Our task is to build a model that predicts the total ride duration of taxi trips in New York City. Our primary dataset is one released by the NYC Taxi and Limousine Commission, which includes pickup time, geo-coordinates, number of passengers, and several other variables.**

# **General Guidelines** : -  

1.   Well-structured, formatted, and commented code is required. 
2.   Exception Handling, Production Grade Code & Deployment Ready Code will be a plus. Those students will be awarded some additional credits. 
     
     The additional credits will have advantages over other students during Star Student selection.
       
             [ Note: - Deployment Ready Code is defined as, the whole .ipynb notebook should be executable in one go
                       without a single error logged. ]

3.   Each and every logic should have proper comments.
4. You may add as many number of charts you want. Make Sure for each and every chart the following format should be answered.
        

```
# Chart visualization code
```
            

*   Why did you pick the specific chart?
*   What is/are the insight(s) found from the chart?
* Will the gained insights help creating a positive business impact? 
Are there any insights that lead to negative growth? Justify with specific reason.

5. You have to create at least 15 logical & meaningful charts having important insights.


[ Hints : - Do the Vizualization in  a structured way while following "UBM" Rule. 

U - Univariate Analysis,

B - Bivariate Analysis (Numerical - Categorical, Numerical - Numerical, Categorical - Categorical)

M - Multivariate Analysis
 ]





6. You may add more ml algorithms for model creation. Make sure for each and every algorithm, the following format should be answered.


*   Explain the ML Model used and it's performance using Evaluation metric Score Chart.


*   Cross- Validation & Hyperparameter Tuning

*   Have you seen any improvement? Note down the improvement with updates Evaluation metric Score Chart.

*   Explain each evaluation metric's indication towards business and the business impact pf the ML model used.




















# ***Let's Begin !***

## ***1. Know Your Data***

### Import Libraries

In [ ]:
#Installing haversine to calulate the distance 
!pip install haversine

In [ ]:
# Import Libraries
import numpy as np
import pandas as pd
import seaborn as sns
import missingno as msno
import matplotlib.pyplot as plt
import warnings; warnings.simplefilter('ignore')
import datetime as dt
%matplotlib inline
import haversine as hs
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor

### Dataset Loading

In [ ]:
#Mount drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Load Dataset
df = pd.read_csv('/content/drive/MyDrive/Capstone Project -  NYC Taxi Trip Time Prediction/NYC Taxi Data.csv')

### Dataset First View

In [ ]:
# Dataset First Look
df.head()

### Dataset Rows & Columns count

In [ ]:
# Dataset Rows & Columns count
print(F'Number of rows in this dataset is {df.shape[0]}')
print(F'Number of columns in this dataset is {df.shape[1]}')

### Dataset Information

In [ ]:
# Dataset Info
df.info()

#### Duplicate Values

In [ ]:
# Dataset Duplicate Value Count
duplicate_value = df[df.duplicated()]
print(f'Duplicate Value Count is {len(duplicate_value)}')

#### Missing Values/Null Values

In [ ]:
# Missing Values/Null Values Count
df.isnull().sum()

In [ ]:
# Visualizing the missing values
msno.bar(df)
plt.show()

### What did you know about your dataset?





From above visualization we can see that their is no  missing and duplicated values.

## ***2. Understanding Your Variables***

In [ ]:
# Dataset Columns
df_columns = df.columns
df_columns

In [ ]:
# Dataset Describe
df.describe()

### Variables Description 

From describe we know about the count, mean, standard deviation, min, max, top 25 percent values, top 50 percent values, top 75 percent values of all numerical columns.  

### Check Unique Values for each variable.

In [ ]:
# Check Unique Values for each variable.
df.nunique()

## 3. ***Data Wrangling***

### Data Wrangling Code

In [ ]:
# Write your code to make your dataset analysis ready.
#changing appropiate dtype of columns
df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])
df['dropoff_datetime'] = pd.to_datetime(df['dropoff_datetime'])

In [ ]:
#Extracting day name from datetime
df['pickup_day'] = df['pickup_datetime'].dt.day_name().astype('category')
df['dropoff_day'] = df['dropoff_datetime'].dt.day_name().astype('category')

In [ ]:
#Extracting weekday from datetime
df['pickup_day_no'] = df['pickup_datetime'].dt.weekday
df['dropoff_day_no'] = df['dropoff_datetime'].dt.weekday

In [ ]:
#Extracting hours from datetime
df['pickup_hrs'] = df['pickup_datetime'].dt.hour.astype(float)
df['dropoff_hrs'] = df['dropoff_datetime'].dt.hour.astype(float)

In [ ]:
#Extracting month from datetime
df['pickup_month'] = df['pickup_datetime'].dt.month.astype(float)
df['dropoff_month'] = df['dropoff_datetime'].dt.month.astype(float)

In [ ]:
#Year
df['pickup_Year']=df['pickup_datetime'].dt.year.astype(float)
df['dropoff_Year']=df['dropoff_datetime'].dt.year.astype(float)

In [ ]:
# Time
df['Pick_Up_time'] = df['pickup_datetime'].dt.time
df['Drop_off_time'] = df['dropoff_datetime'].dt.time

In [ ]:
#Date
df['pickup_date']=df['pickup_datetime'].dt.date
df['dropoff_date']=df['dropoff_datetime'].dt.date

In [ ]:
#Function for calculating the distance betwwen pickup and dropoff (latitude,longitude)
def distance(df):
  lat1,lon1,lat2,lon2 = df['pickup_latitude'],df['pickup_longitude'],df['dropoff_latitude'],df['dropoff_longitude']
  a = hs.haversine((lat1,lon1),(lat2,lon2))
  return a

In [ ]:
df['distance'] = df.apply(lambda x : distance(x),axis=1)

In [ ]:
#Calculating speed of taxi
df['speed'] = (df['distance']*3600)/df['trip_duration']

In [ ]:
# Function for time of day
def time_of_day(x):
  if x in range(6,12):
    return 'Morning'
  elif x in range(12,16):
    return 'Afternoon'
  elif x in range(16,22):
    return 'Evening'
  else:
    return 'Late night' 

In [ ]:
#Now let us apply this function and create new columns in the dataset.
df['pickup_time_of_day'] = df['pickup_hrs'].apply(time_of_day)
df['dropoff_time_of_day'] = df['dropoff_hrs'].apply(time_of_day)

In [ ]:
df.head()

### What all manipulations have you done and insights you found?



1.   Changed the dtype of columns 'pickup_datetime' and 'dropoff_datetime' to datetime dtype.
2.   Adding important columns 'pickup_day', 'dropoff_day', 'pickup_day_no', 'dropoff_day_no', 'pickup_hrs', 'dropoff_hrs', 'pickup_month', 'dropoff_month', 'Pick_Up_time','Year', 'Date'. 



## ***4. Data Vizualization, Storytelling & Experimenting with charts : Understand the relationships between variables***

#### Pie chart for vendor ID distribution 

In [ ]:
# Pie chart for vendor ID distribution 
df['vendor_id'].value_counts().plot.pie(figsize = (7,9), autopct='%1.1f%%')
plt.legend()
plt.show()

##### 1. Why did you pick the specific chart?

I picked up pie chart because in this we can see the exact distribution in percentage. 

##### 2. What is/are the insight(s) found from the chart?

We can observe that there are approximately similar percentage on both side

##### 3. Will the gained insights help creating a positive business impact? 
Are there any insights that lead to negative growth? Justify with specific reason.

No

#### Countplot for Passenger Count

In [ ]:
'''Countplot for Passenger Count'''

#Plotting Sub Plot 
fig, axes = plt.subplots(1, 2,figsize=(20,5))

#Plotting bar graph 
sns.countplot(df['passenger_count'],ax=axes[0])
plt.title('Distribution of passenger count ')
plt.xlabel('Passenger')
plt.ylabel('Count')

#Plotting Box Plot
sns.boxplot(df['passenger_count'],color='blue',ax=axes[1])

plt.show()

##### 1. Why did you pick the specific chart?

I picked up countplot and boxplot.
Countplot is to see the count of unique passenger and boxplot to see the outliers.

##### 2. What is/are the insight(s) found from the chart?


According to Driver Rule 54-15(g) of 'NYC Taxi and Limousine Commission' , maximum permissible number of passengers in a taxi is 6. Hence we will drop values which are greater than 6.

##### 3. Will the gained insights help creating a positive business impact? 
Are there any insights that lead to negative growth? Justify with specific reason.

Yes, the gained insights help creating a positive business impact.

There are some values that are represented by 0, we will drop these values because this is clearly a mistake that lead to negative growth



#### Trip duration overall distribution

In [ ]:
'''trip duration overall distribution '''

# taking the 97% Quantile values to remove outliers 
tripduration = df[df.trip_duration < df.trip_duration.quantile(.97)]

#Plotting Trip duration VS Trip_duration.counts
tripduration.groupby('trip_duration').count()['id'].plot()
plt.xlabel('trip duration (sec)')
plt.ylabel('trips counts')
plt.title('Duration Distribution')
plt.show()

##### 1. Why did you pick the specific chart?

I picked line plot because large numbers of rows and in this i can easily see the distribution.

##### 2. What is/are the insight(s) found from the chart?

I found most of the peoples take taxi trip between 400 to 1000 sec. 

##### 3. Will the gained insights help creating a positive business impact? 
Are there any insights that lead to negative growth? Justify with specific reason.

Yes, the gained insights help creating a positive business impact.

Yes, Some trips take too long/too fast are obvious wrong, maybe because of tech/maunal operation issues, should filter them in following process


#### Plotting bar graph for store_and_fwd_flag

This flag indicates whether the trip record was held in vehicle memory before sending to the vendor because the vehicle did not have a connection to the server - Y=store and forward; N=not a store and forward trip

In [ ]:
#Plotting bar graph for store_and_fwd_flag
sns.countplot(df['store_and_fwd_flag'])
plt.title('Distribution of store and flag values ')
plt.xlabel('store and flag values')
plt.ylabel('Count')
plt.show()

##### 1. Why did you pick the specific chart?

I picked bar graph to see the count of N and Y.

##### 2. What is/are the insight(s) found from the chart?

I found their are almost 100 percent values is N means not a store and forward trip.  

##### 3. Will the gained insights help creating a positive business impact? 
Are there any insights that lead to negative growth? Justify with specific reason.

No

#### Plotting pie chart passenger_count

In [ ]:
#Plotting pie chart passenger_count
df['passenger_count'].value_counts().plot.pie(figsize = (7,9), autopct='%1.1f%%')
plt.legend()

plt.show()


##### 1. Why did you pick the specific chart?

I picked pie chart to see the percentage distribution of passenger count.

##### 2. What is/are the insight(s) found from the chart?

I found their are some passenger with zero percent distribution.

##### 3. Will the gained insights help creating a positive business impact? 
Are there any insights that lead to negative growth? Justify with specific reason.

Yes, the gained insights help creating a positive business impact.

There are some values that are represented by 0, we will drop these values because this is clearly a mistake that lead to negative growth

#### Plotting graph of Pickup and Dropoff counts wr to dattes.

In [ ]:
'''Plotting graph of Pickup and Dropoff counts wr to dattes.'''
fig, ax = plt.subplots(ncols=2, sharey=True)
fig.set_size_inches(12, 5)

#Plotting for Pickups
ax[0].plot(df.groupby('pickup_date').count()['id'], 'go-', alpha=0.5)
ax[0].set(xlabel='date', ylabel='Count',title="pickup counts")

#PLotting For Dropoffs
ax[1].plot(df.groupby('dropoff_date').count()['id'], 'bo-', alpha=0.5)
ax[1].set(xlabel='date', ylabel='Count',title="dropoff counts")

plt.show()

##### 1. Why did you pick the specific chart?

I picked line plot because large numbers of rows and in this i can easily see the distribution.

##### 2. What is/are the insight(s) found from the chart?

We observe that daily Drop Off vary between 6000 and 1000, values on 26-07-01,2016-01-23,2016-01-24 are considerably low.

##### 3. Will the gained insights help creating a positive business impact? 
Are there any insights that lead to negative growth? Justify with specific reason.

No

#### Variation of passenger count with pickupdate 

In [ ]:
#Variation of passenger count with pickupdate 
df.groupby(['pickup_date']).sum()['passenger_count'].plot()

plt.show()

##### 1. Why did you pick the specific chart?

I picked line plot to see the variation of passengeer count with pickup date.

##### 2. What is/are the insight(s) found from the chart?

I found in between 2016-01 to 2016-02 ,passenger count was low as comparison to other dates.

##### 3. Will the gained insights help creating a positive business impact? 
Are there any insights that lead to negative growth? Justify with specific reason.

Yes, the gained insights help creating a positive business impact.

There are no such insights that lead to negative growth.

#### Variation of passenger count with dropoff date 

In [ ]:
#Variation of passenger count with dropoff date 
df.groupby(['dropoff_date']).sum()['passenger_count'].plot()

plt.show()

##### 1. Why did you pick the specific chart?

I picked line plot to see the variation of passengeer count with dropoff date.

##### 2. What is/are the insight(s) found from the chart?

I found in between 2016-01 to 2016-02 and 2016-07 ,passenger count was low as comparison to other dates.

##### 3. Will the gained insights help creating a positive business impact? 
Are there any insights that lead to negative growth? Justify with specific reason.

Yes, the gained insights help creating a positive business impact.

There are no such insights that lead to negative growth.

#### Plotting a pivot table with pickup date time hour and aggrgate (mean median and std) of trip duration 

In [ ]:
''' Plotting a pivot table with pickup date time hour and aggrgate (mean median and std) of trip duration '''

pd.pivot_table(tripduration, index='pickup_hrs' ,aggfunc=np.mean)['trip_duration'].plot(label='mean')
pd.pivot_table(tripduration, index='pickup_hrs' ,aggfunc=np.median)['trip_duration'].plot(label='median')
pd.pivot_table(tripduration, index='pickup_hrs' ,aggfunc=np.std)['trip_duration'].plot(label='std')

# plotting the table 
plt.legend(loc=0)
plt.xlabel('Pick up Hour (0-23)')
plt.ylabel('order counts ')
plt.title('order counts VS Pickup hour')
plt.show()

##### 1. Why did you pick the specific chart?

I picked line plot to see comparion of mean , median and mode of pickup hour.

##### 2. What is/are the insight(s) found from the chart?

I found mean and median of pickup hour is minimum at 6.

##### 3. Will the gained insights help creating a positive business impact? 
Are there any insights that lead to negative growth? Justify with specific reason.

Yes, the gained insights help creating a positive business impact.

There are no such insights that lead to negative growth.

#### Plotting  store_and_fwd_flag VS trip_duration

In [ ]:
# Plotting  store_and_fwd_flag VS trip_duration
plt.figure(figsize=(14,6))
sns.barplot(x='pickup_hrs',y='trip_duration',data=df,hue='store_and_fwd_flag')
plt.xlabel('Pickup Hour',fontsize=16)
plt.ylabel('Trip Duration',fontsize=16)
plt.show()

##### 1. Why did you pick the specific chart?

I picked barplot to see the comparison of store_and_fwd_flag VS trip_duration.

##### 2. What is/are the insight(s) found from the chart?

I found store_and_fwd_flag is almost equally distributed in trip duration.

##### 3. Will the gained insights help creating a positive business impact? 
Are there any insights that lead to negative growth? Justify with specific reason.

Yes, the gained insights help creating a positive business impact.

There are no such insights that lead to negative growth.

#### Plot showing variation in trip duraion for diffrent pickups for diffrent months 

In [ ]:
#Plot showing variation in trip duraion for diffrent pickups for diffrent months 
plt.figure(figsize=(14,6))
sns.pointplot(x='pickup_hrs',y='trip_duration',data=tripduration,hue='pickup_month')
plt.xlabel('Pickup Hour',fontsize=16)
plt.ylabel('Trip Duration)',fontsize=16)
plt.show()

##### 1. Why did you pick the specific chart?

I pickup point plot to see variation in trip duraion for diffrent pickups for diffrent months.

##### 2. What is/are the insight(s) found from the chart?

I found in trip duraion for diffrent pickups for diffrent months is minimum at 6.

##### 3. Will the gained insights help creating a positive business impact? 
Are there any insights that lead to negative growth? Justify with specific reason.

Yes, the gained insights help creating a positive business impact.

There are no such insights that lead to negative growth.

#### Plot showing variation in trip duraion for diffrent dropoffs for diffrent months

In [ ]:
#Plot showing variation in trip duraion for diffrent dropoffs for diffrent months 
plt.figure(figsize=(14,6))
sns.pointplot(x='dropoff_hrs',y='trip_duration',data=tripduration,hue='pickup_month')
plt.xlabel('dropoff Hour',fontsize=16)
plt.ylabel('Trip Duration)',fontsize=16)
plt.show()

##### 1. Why did you pick the specific chart?

I picked point plot to see variation in trip duraion for diffrent dropoffs for diffrent months.

##### 2. What is/are the insight(s) found from the chart?

Months only affect duration a little bit ; while pickup hours seems may be infulence that more.

##### 3. Will the gained insights help creating a positive business impact? 
Are there any insights that lead to negative growth? Justify with specific reason.

Yes, the gained insights help creating a positive business impact.

There are no such insights that lead to negative growth.

#### Plotting a line graph to represent the change in time duration of trip for diffrent days of the week

In [ ]:
# Plotting a line graph to represent the change in time duration of trip for diffrent days of the week 
plt.figure(figsize=(14,6))
sns.pointplot(x='pickup_hrs',y='trip_duration',data=tripduration,hue='pickup_day')
plt.xlabel('Pickup Hour',fontsize=16)
plt.ylabel('Trip Duration)',fontsize=16)
plt.show()

##### 1. Why did you pick the specific chart?

I pikcked point plot to to represent the change in time duration of trip for diffrent days of the week.

##### 2. What is/are the insight(s) found from the chart?



*   During the moring rush hours from Monday to Friday ie. 7 AM to 10 AM, maybe owing to traffic jam.
*   Duration are high during workday (Mon.-Fri.)



##### 3. Will the gained insights help creating a positive business impact? 
Are there any insights that lead to negative growth? Justify with specific reason.

Yes, the gained insights help creating a positive business impact.

There are no such insights that lead to negative growth.

#### Correlation Heatmap

In [ ]:
#Let's see correlation graph
plt.figure(figsize=(20,12))
correlation = df.corr()
sns.heatmap(abs(correlation), annot=True, cmap='coolwarm')
plt.show()

##### 1. Why did you pick the specific chart?

I picked heatmap to see the correlation.

##### 2. What is/are the insight(s) found from the chart?

I found there are some variables whose correlation is too high, so we will remove that variables.

##***5. Feature Engineering & Data Pre-processing***

### 1. Handling Missing Values

In [ ]:
# Handling Missing Values & Missing Value Imputation
df.isnull().sum()

#### What all missing value imputation techniques have you used and why did you use those techniques?

In this dataset their is no missing values.

### 2. Handling Outliers

In [ ]:
# Handling Outliers & Outlier treatments

#Taking numerical columns
numeric_col = df.describe().columns

#checking outliers of numeric columns
for i in numeric_col:
  sns.boxplot(df[i])
  plt.show()

In [ ]:
#Function to check outliers
def check_outliers(start,end,step,column):
  for i in range(start,end,step):
    a = column.values
    a=np.sort(a)
    print('{} percentile value is {}'.format(i,int(column.quantile(i/100))))
  print('{} percentile value is {}'.format(100,a[-1]))

In [ ]:
#Removing outliers of column trip duration
check_outliers(0,100,10,df['trip_duration'])

In [ ]:
check_outliers(90,100,1,df['trip_duration'])

In [ ]:
df = df.drop(df[df['trip_duration']>=3440].index)

In [ ]:
sns.boxplot(df.trip_duration)
plt.show()

In [ ]:
#Removing outliers from passenger count
df['passenger_count'].value_counts()

In [ ]:
df = df.drop(df[df['passenger_count']==0].index)
df = df.drop(df[df['passenger_count']>=7].index)

In [ ]:
sns.boxplot(df.passenger_count)
plt.show()

In [ ]:
#Removing outliers of distance column
check_outliers(0,100,10,df['distance'])

In [ ]:
check_outliers(90,100,1,df['distance'])

In [ ]:
df = df.drop(df[df['distance']==0].index)
df = df.drop(df[df['distance']>20].index)

In [ ]:
sns.boxplot(df.distance)
plt.show()

In [ ]:
#Removing outliers of speed column
check_outliers(0,100,10,df['speed'])

In [ ]:
check_outliers(90,100,1,df['speed'])

In [ ]:
df = df[df['speed'] <= 40]
df = df[df['speed'] >= 5]

In [ ]:
sns.boxplot(df['speed'])
plt.show()

##### What all outlier treatment techniques have you used and why did you use those techniques?

I used 2 techniques to check outliers boxplot and percentile method because in percentile i can easily see the outliers from which points outliers started.

### 3. Categorical Encoding

In [ ]:
# Encode your categorical columns
df1 = pd.get_dummies(df , columns=['store_and_fwd_flag','pickup_time_of_day','dropoff_time_of_day'],prefix=['store','pickup','dropoff'])

#### What all categorical encoding techniques have you used & why did you use those techniques?

I have created dummies of categorial columns 'store_and_fwd_flag','pickup_time_of_day','dropoff_time_of_day'.

### 4. Feature Manipulation & Selection

#### 1. Feature Manipulation

In [ ]:
# Manipulate Features to minimize feature correlation and create new features
#Let's see correlation graph
plt.figure(figsize=(20,12))
correlation = df1.corr()
sns.heatmap(abs(correlation), annot=True, cmap='coolwarm')
plt.show()

In [ ]:
df_corr = df1.copy()
df_corr.columns

#### 2. Feature Selection

In [ ]:
# Select your features wisely to avoid overfitting
#Lets drop some feature which are having high correlation (more than 0.4)
df_corr.drop(['dropoff_Afternoon','dropoff_Evening', 'dropoff_Late night', 'dropoff_Morning','store_Y','store_N','dropoff_day_no',
              'pickup_Evening','pickup_Morning','dropoff_month','dropoff_hrs', 'id'],axis=1,inplace=True)

In [ ]:
df_corr.columns

In [ ]:
#Lets check correlation graph again after removing high correlation
plt.figure(figsize=(20,12))
correlation = df_corr.corr()
sns.heatmap(abs(correlation), annot=True, cmap='coolwarm')
plt.show()

##### What all feature selection methods have you used  and why?

I used heatmap for all feature selection to check the correlation.

##### Which all features you found important and why?

I found 'vendor_id', 'pickup_datetime', 'dropoff_datetime', 'passenger_count',
       'pickup_longitude', 'pickup_latitude', 'dropoff_longitude',
       'dropoff_latitude', 'trip_duration', 'pickup_day', 'dropoff_day',
       'pickup_day_no', 'pickup_hrs', 'pickup_month', 'pickup_Year',
       'dropoff_Year', 'Pick_Up_time', 'Drop_off_time', 'pickup_date',
       'dropoff_date', 'distance', 'speed', 'pickup_Afternoon',
       'pickup_Late night' these features are important because they all have correlation less than 0.4 .

### 5. Data Transformation

In [ ]:
#Spliting dependent and independent variables
independent_variables=['pickup_longitude', 'pickup_latitude','dropoff_longitude', 'dropoff_latitude','distance', 'pickup_hrs']

dependent_variables = 'trip_duration'

In [ ]:
X = df_corr[independent_variables]

y = df_corr[dependent_variables]

In [ ]:
print(X.shape)
print(y.shape)

#### Do you think that your data needs to be transformed? If yes, which transformation have you used. Explain Why?

In [ ]:
# Transform Your data
scaler = StandardScaler()
X = scaler.fit_transform(X)

### 6. Data Scaling

In [ ]:
# Scaling your data
X[0:2]

##### Which method have you used to scale you data and why?

### 7. Dimesionality Reduction

##### Do you think that dimensionality reduction is needed? Explain Why?

Yes, dimensionality reduction is needed to find the best parameters.

##### Which dimensionality reduction technique have you used and why? (If dimensionality reduction done on dataset.)

I used RandomizedSearchCV for dimensionality reduction because gridsearchcv take too much time.

### 8. Data Splitting

In [ ]:
# Split your data to train and test. Choose Splitting ratio wisely.

# Splitting the dataset into the Training set and Test set
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [ ]:
print('Train Data Shape')
print(X_train.shape)
print(Y_train.shape)
print('\n')
print('Test Data Shape')
print(X_test.shape)
print(Y_test.shape)

##### What data splitting ratio have you used and why? 

I used 70% data for training and 30% of data for test because it gives the best results.

## ***6. ML Model Implementation***

We will start by building simple prediction models and work our way upto more complex models. but first lets start by defining some important function's.




### Defining function to fit model

In [ ]:
#Defining function to fit model

def fit_mod(model, X_train, X_test, Y_train, Y_test):

  '''Takes regressor model and train test splits as input and prints the
  evaluation matrices with the plot and returns the model'''
  global Y_pred
  # Fitting the model
  model.fit(X_train,Y_train)
  Y_pred = model.predict(X_test)
  a,p = Y_test,Y_pred 

  return Y_pred

### Defining function to get Evaluation matrix

In [ ]:
#Defining to get Evaluation matrix

def plot_model(model, X_train, X_test, y_train, y_test):

# Plotting actual and predicted values and the feature importances:
  plt.figure(figsize=(18,3))
  plt.subplot(1,1,1)
  plt.plot((Y_pred)[:100])
  plt.plot((np.array(y_test)[:100]))
  plt.legend(["Predicted","Actual"])
  plt.title('Actual and Predicted trip duration')

  return model

### Defining function to get Get_model

In [ ]:
'''Storing Values in a list'''
#TEST 
test_mse= []
test_r2=[]
test_adjr2=[]
test_rmse=[]
#TRAIN
train_mse=[]
train_r2=[]
train_adjr2=[]
train_rmse=[]

def Get_model(model,X_train,X_test,Y_train,Y_test):
      
    fit_mod(model, X_train, X_test, Y_train, Y_test)
    ''' Storing predictions for Evaluation '''
    Y_pred_train = model.predict(X_train)
    Y_pred_test = model.predict(X_test)

    '''Computing MSE ,R2 and Adjusted R2 score for train data''' 
    # Mean Square Error 
    train_mse_  = mean_squared_error(Y_train, Y_pred_train)
    print("Train MSE :" , train_mse_)

    #Root mean square error 
    train_rmse_ = np.sqrt(train_mse_)
    print("Train RMSE :" ,train_rmse_)

    #R2
    train_r2_ = r2_score(Y_train, Y_pred_train)
    print("Train R2 :" ,train_r2_)

    # Adjusted R2
    train_ar2_= 1-(1-r2_score((Y_train), (Y_pred_train)))*((X_train.shape[0]-1)/(X_train.shape[0]-X_train.shape[1]-1))
    print("Train Adjusted R2 : ",train_ar2_)

    '''Computing MSE ,R2 and Adjusted R2 score for test data''' 
    # Mean Square Error 
    test_mse_  = mean_squared_error(Y_test, Y_pred_test)
    print("Test MSE :" , test_mse_)
    #Root mean square error 
    test_rmse_ = np.sqrt(test_mse_)
    print("Test RMSE :" ,test_rmse_)

    #R2
    test_r2_ = r2_score(Y_test, Y_pred_test)
    print("Test R2 :" ,test_r2_)

    # Adjusted R2
    test_ar2_= 1-(1-r2_score((Y_test), (Y_pred_test)))*((X_test.shape[0]-1)/(X_test.shape[0]-X_test.shape[1]-1))
    print("Test Adjusted R2 : ", test_ar2_)

    '''Storing Values in a list '''
    model=model
    #TEST 
    test_mse.append(test_mse_)
    test_r2.append(test_r2_)
    test_adjr2.append(test_ar2_)
    test_rmse.append(test_rmse_)
    #TRAIN
    train_mse.append(train_mse_)
    train_r2.append(train_r2_)
    train_adjr2.append(train_ar2_)
    train_rmse.append(train_rmse_)

    try:
      plot_model(model, X_train, X_test, Y_train, Y_test)
    except Exception:
      pass    

    return     

### ML Model - Linear regression

In [ ]:
# Defining LinearRegression and fitting the function 
lr = LinearRegression()
Get_model(lr, X_train, X_test, Y_train, Y_test)

### ML Model - Decision Tree

In [ ]:
# Minimum number of samples required to split a node:
min_samples_split = [10,20,30]

# Minimum number of samples required at each leaf node:
min_samples_leaf = [8,16,22]

# Maximum depth of trees:
max_depth = [6,8,10,15]

Commenting this because it take to much time to run.

In [ ]:
'''
#Defining Model 
regressor = DecisionTreeRegressor(random_state=1)

# Hyperparameter Grid
grid = {'max_depth' : max_depth,
        'min_samples_split' : min_samples_split,
        'min_samples_leaf' : min_samples_leaf}

# GridSearch to find the best parameters
dtr = RandomizedSearchCV(regressor, param_distributions = grid, scoring = 'neg_mean_squared_error', cv=5,random_state=6,verbose=True,n_jobs=-1)

#Fitting the model 
Get_model(dtr, X_train, X_test, Y_train, Y_test)
'''

In [ ]:
'''
print('The best fit alpha value is found out to be :' ,dtr.best_params_)
print('The R2 score using the same alpha is :', dtr.best_score_)
'''

### ML Model - Random Forest

In [ ]:
# Number of trees in random forest
n_estimators = [50,100,150]

# Number of features to consider at every split
max_features = ['auto', 'sqrt']

# Maximum number of levels in tree
max_depth = [10,15]

# Minimum number of samples required to split a node
min_samples_split = [5, 10,15]

# Minimum number of samples required at each leaf node
min_samples_leaf = [5, 10,15]

Commenting this because it take to much time to run.

In [ ]:
#Fitting model
''' 
regressor = RandomForestRegressor(random_state=2,oob_score=False,verbose=True)


# Hyperparameter Grid
grid = {'n_estimators' : n_estimators,
        'max_depth' : max_depth,
        'min_samples_split' : min_samples_split,
        'min_samples_leaf' : min_samples_leaf}

# GridSearch to find the best parameters
rf = RandomizedSearchCV(regressor, param_distributions = grid, scoring = 'neg_mean_squared_error', cv=2,random_state=6,verbose=True,n_jobs=-1)

Get_model(rf, X_train, X_test, Y_train, Y_test)
'''

In [ ]:
'''
print('The best fit alpha value is found out to be :' ,rf.best_params_)
print('The R2 score using the same alpha is :', rf.best_score_)
'''

### ML Model - XG boost

In [ ]:
# Learning rate:
eta = [0.05,0.08,0.1]

# HYperparameter Grid
grid = {'learning_rate': [0.1, 0.2, 0.3],
        'max_depth': [5, 8, 10],
        'min_samples_' : [2,4,6],
        'n_estimators' : [100,200,300]}

In [ ]:
#Regressor
'''
regressor = XGBRegressor(silent=True,random_state=0, objective='reg:squarederror')

# GridSearch to find the best parameters
xgb = RandomizedSearchCV(regressor, param_distributions = grid, scoring = 'neg_mean_squared_error', cv=2,random_state=6,verbose=True,n_jobs=-1)

#Fitting the model 
Get_model(xgb, X_train, X_test, Y_train, Y_test)
'''

In [ ]:
'''
print('The best fit alpha value is found out to be :' ,xgb.best_params_)
print('The R2 score using the same alpha is :', xgb.best_score_)
'''

### Model Evalutaion

In [ ]:
models= ['Linear Regression','DecisionTree Regressor','Ramdom Forest','XGBoost Regressor']
with pd.option_context('display.precision', 10):
 model_comparison = pd.DataFrame({'Model Name': models,
                          'Train_MSE': train_mse,'Test_MSE': test_mse,
                          'Train_R^2': train_r2, 'Test_R^2': test_r2,
                          'Train_Adjusted_R^2': train_adjr2, 'Test_Adjusted_R^2': test_adjr2,
                          'Test Rsme': test_rmse,'Train Rsme': train_rmse})
model_comparison

In [ ]:
# Writing the dataframe to a csv to ensure no data loss in working
model_comparison.to_csv('/content/drive/MyDrive/Capstone Project -  NYC Taxi Trip Time Prediction/model.csv')

In [ ]:
# Reading the dataframe from the csv file again to import the scraped data from the already prepared csv file
model_comparison = pd.read_csv('/content/drive/MyDrive/Capstone Project -  NYC Taxi Trip Time Prediction/model.csv')

In [ ]:
model_comparison.drop(['Unnamed: 0'],axis=1,inplace=True)
model_comparison

### 1. Which Evaluation metrics did you consider for a positive business impact and why?

 We can use XGBoost Regressor evaluation metrices for a positive business impact because in this evaluation metrices we get best performance.

### 2. Which ML model did you choose from the above created models as your final prediction model and why?

XGBoost Regressor	model is final prediction model from above models because in this model i get best r2 score. 

# **Conclusion**






*   We can see that MSE and R2 and Adjusted R2 which are the metrics used to evaluate the performance of regression model of Linear Regression, Decision Tree, Random Forest and XGBoost Regressor.
*   The Linear models don't show good performance on our training and testing environment.


*   There is a significant improvement in the RMSE score for the tuned XGBoost regressor over the Random Forest regressor when trained on the feature selection group.
*   The variance score of the XGB Regression more efficient than other Regression Models.


*   XGBoost Regression model proved to be much more efficient in predicting the output. But it takes much more time to train it over the large dataset with more complexity as compared to the other Regresson models.






*   From above table we can conclude that XGBoost Regression (80%) is the best models as compare to the other models to predict the trip duration for a particular taxi.






### ***Hurrah! You have successfully completed your Machine Learning Capstone Project !!!***